# Risk Management and Portfolio Optimization

This notebook demonstrates advanced portfolio optimization techniques and risk management strategies using the Factor Lab.

## What You'll Learn:
- Advanced portfolio optimization methods (Black-Litterman, Risk Parity)
- Risk budgeting and factor risk models
- Dynamic hedging strategies
- Stress testing and scenario analysis
- Portfolio risk attribution

In [12]:
# Import required libraries
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

from factor_lab import (
    DataManager, FactorCalculator, PortfolioOptimizer,
    Backtester, ChartManager, PerformanceAnalyzer
)

print("✅ Factor Lab imported successfully!")

✅ Factor Lab imported successfully!


## 1. Initialize Components and Data

In [2]:
start_date="2020-01-01"
end_date="2024-01-01"

# Initialize components
data_manager = DataManager()
factor_calc = FactorCalculator()
# Note: PortfolioOptimizer will be initialized after returns data is available
backtester = Backtester(start_date, end_date)
charts = ChartManager()
performance = PerformanceAnalyzer()

# Define diversified universe across sectors
universe = {
    'Technology': ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'META', 'ORCL'],
    'Healthcare': ['JNJ', 'PFE', 'UNH', 'ABBV', 'MRK', 'TMO'],
    'Finance': ['JPM', 'BAC', 'WFC', 'GS', 'MS', 'C'],
    'Consumer': ['PG', 'KO', 'WMT', 'HD', 'MCD', 'NKE'],
    'Industrial': ['BA', 'CAT', 'GE', 'MMM', 'HON', 'UPS'],
    'Energy': ['XOM', 'CVX', 'COP', 'SLB', 'EOG', 'PSX']
}

# Flatten universe
all_symbols = [symbol for sector_symbols in universe.values() for symbol in sector_symbols]

print(f"📊 Universe: {len(all_symbols)} stocks across {len(universe)} sectors")
for sector, symbols in universe.items():
    print(f"   {sector}: {len(symbols)} stocks")

📊 Universe: 36 stocks across 6 sectors
   Technology: 6 stocks
   Healthcare: 6 stocks
   Finance: 6 stocks
   Consumer: 6 stocks
   Industrial: 6 stocks
   Energy: 6 stocks


In [3]:
# Fetch price data
print("📈 Fetching price data...")
prices = data_manager.get_prices(
    symbols=all_symbols,
    start_date="2020-01-01",
    end_date="2024-01-01"
)

# Calculate returns
returns = prices.pct_change().dropna()

print(f"✅ Data loaded: {len(prices)} price observations")
print(f"📅 Date range: {prices.index[0].strftime('%Y-%m-%d')} to {prices.index[-1].strftime('%Y-%m-%d')}")
print(f"📊 Available symbols: {len(prices.columns)}")

# Clean returns data for optimization
print("\n🧹 Cleaning returns data...")
# Remove symbols with insufficient data or extreme values
min_observations = int(0.8 * len(returns))  # Require 80% of observations
valid_symbols = []
for symbol in returns.columns:
    symbol_returns = returns[symbol].dropna()
    if (len(symbol_returns) >= min_observations and 
        symbol_returns.std() > 0 and 
        np.isfinite(symbol_returns).all()):
        valid_symbols.append(symbol)

clean_returns = returns[valid_symbols].dropna()
print(f"✅ Cleaned data: {len(valid_symbols)} symbols, {len(clean_returns)} observations")

# Display basic statistics
print("\n📋 Return Statistics:")
print(f"   Mean daily return: {clean_returns.mean().mean():.4f}")
print(f"   Mean daily volatility: {clean_returns.std().mean():.4f}")
if len(clean_returns.columns) > 1:
    corr_values = clean_returns.corr().values[np.triu_indices_from(clean_returns.corr().values, k=1)]
    print(f"   Correlation range: {corr_values.min():.3f} to {corr_values.max():.3f}")

# Store clean returns for later use in optimization
returns = clean_returns
print(f"\n✅ Returns data prepared for optimization ({len(valid_symbols)} symbols)")
print("📝 Note: PortfolioOptimizer will be initialized when needed in optimization section")

📈 Fetching price data...
✅ Data loaded: 1006 price observations
📅 Date range: 2020-01-02 to 2023-12-29
📊 Available symbols: 36

🧹 Cleaning returns data...
✅ Cleaned data: 36 symbols, 1005 observations

📋 Return Statistics:
   Mean daily return: 0.0007
   Mean daily volatility: 0.0223
   Correlation range: 0.123 to 0.910

✅ Returns data prepared for optimization (36 symbols)
📝 Note: PortfolioOptimizer will be initialized when needed in optimization section
✅ Data loaded: 1006 price observations
📅 Date range: 2020-01-02 to 2023-12-29
📊 Available symbols: 36

🧹 Cleaning returns data...
✅ Cleaned data: 36 symbols, 1005 observations

📋 Return Statistics:
   Mean daily return: 0.0007
   Mean daily volatility: 0.0223
   Correlation range: 0.123 to 0.910

✅ Returns data prepared for optimization (36 symbols)
📝 Note: PortfolioOptimizer will be initialized when needed in optimization section


## 2. Factor Calculation and Risk Model

In [4]:
print("🔢 Building factor risk model...")

# Calculate factor exposures
factor_exposures = pd.DataFrame(index=prices.index)

# Market factor (beta to SPY)
spy_data = data_manager.get_prices(['SPY'], start_date="2020-01-01", end_date="2024-01-01")
if not spy_data.empty:
    spy_returns = spy_data['SPY'].pct_change().dropna()
    
    for symbol in all_symbols:
        if symbol in returns.columns:
            # Calculate rolling 60-day beta
            rolling_beta = returns[symbol].rolling(60).cov(spy_returns) / spy_returns.rolling(60).var()
            factor_exposures[f"{symbol}_market_beta"] = rolling_beta

# Size factor (market cap proxy using price)
for symbol in all_symbols:
    if symbol in prices.columns:
        # Use log price as size proxy (in practice, use actual market cap)
        log_price = np.log(prices[symbol])
        size_factor = log_price.rolling(60).rank(pct=True)  # Percentile rank
        factor_exposures[f"{symbol}_size"] = size_factor

# Momentum factor
prices_data = prices  # Assuming 'prices' is a DataFrame with the necessary price data
momentum_factor = factor_calc.momentum(prices_data, lookback=252)
for symbol in all_symbols:
    if symbol in momentum_factor.columns:
        factor_exposures[f"{symbol}_momentum"] = momentum_factor[symbol]

# Volatility factor
for symbol in all_symbols:
    if symbol in returns.columns:
        volatility = returns[symbol].rolling(60).std() * np.sqrt(252)
        factor_exposures[f"{symbol}_volatility"] = volatility

# Sector exposures (dummy variables)
for sector, sector_symbols in universe.items():
    for symbol in sector_symbols:
        if symbol in prices.columns:
            factor_exposures[f"{symbol}_sector_{sector.lower()}"] = 1.0

factor_exposures = factor_exposures.fillna(0)

print(f"✅ Factor exposures calculated: {factor_exposures.shape[1]} total exposures")
print(f"📊 Factors: Market Beta, Size, Momentum, Volatility, Sector")

# Display sample exposures
sample_cols = [c for c in factor_exposures.columns if 'AAPL' in c][:5]
if sample_cols:
    print("\n📋 Sample factor exposures (AAPL):")
    print(factor_exposures[sample_cols].tail())

🔢 Building factor risk model...
✅ Factor exposures calculated: 180 total exposures
📊 Factors: Market Beta, Size, Momentum, Volatility, Sector

📋 Sample factor exposures (AAPL):
            AAPL_market_beta  AAPL_size  AAPL_momentum  AAPL_volatility  \
Date                                                                      
2023-12-22          0.859863   0.833333      43.729881         0.158783   
2023-12-26          0.860588   0.783333      46.811680         0.159102   
2023-12-27          0.870181   0.783333      47.299887         0.156888   
2023-12-28          0.886559   0.816667      49.705486         0.155606   
2023-12-29          0.889689   0.733333      53.606911         0.155939   

            AAPL_sector_technology  
Date                                
2023-12-22                     1.0  
2023-12-26                     1.0  
2023-12-27                     1.0  
2023-12-28                     1.0  
2023-12-29                     1.0  
✅ Factor exposures calculated: 180 tot

## 3. Portfolio Optimization Methods

In [5]:
# Portfolio optimization comparison
print("💼 Comparing portfolio optimization methods...")

# Use recent data for optimization
lookback_period = 252  # 1 year
optimization_date = returns.index[-60]  # 60 days before end
hist_returns = returns.loc[optimization_date - pd.Timedelta(days=lookback_period*2):optimization_date]

print(f"📅 Optimization date: {optimization_date.strftime('%Y-%m-%d')}")
print(f"📊 Historical data: {len(hist_returns)} observations")
print(f"📊 Available symbols: {len(hist_returns.columns)}")

# Filter symbols with sufficient data - be more lenient
min_obs_required = max(100, int(lookback_period * 0.5))  # At least 100 obs or 50% of lookback
valid_symbols = []
for symbol in hist_returns.columns:
    symbol_data = hist_returns[symbol].dropna()
    if len(symbol_data) >= min_obs_required:
        valid_symbols.append(symbol)

hist_returns_clean = hist_returns[valid_symbols].dropna()

print(f"✅ Valid symbols for optimization: {len(valid_symbols)}")
print(f"📊 Clean data shape: {hist_returns_clean.shape}")

if len(valid_symbols) < 5:
    print("❌ Insufficient symbols for optimization - need at least 5")
    print("📊 Let's use a simpler approach with available data...")
    
    # Use all available data from clean_returns
    hist_returns_clean = clean_returns.tail(200)  # Use last 200 days
    valid_symbols = hist_returns_clean.columns.tolist()
    print(f"📊 Using {len(valid_symbols)} symbols with {len(hist_returns_clean)} observations")

# Use a manageable subset to avoid numerical issues
if len(valid_symbols) > 15:
    # Use top 15 symbols by data completeness
    symbol_quality = hist_returns_clean.count().sort_values(ascending=False)
    top_symbols = symbol_quality.head(15).index.tolist()
    hist_returns_clean = hist_returns_clean[top_symbols]
    valid_symbols = top_symbols
    print(f"📊 Using top {len(valid_symbols)} symbols for optimization")

# ROBUST DATA CLEANING FOR OPTIMIZATION
print("\n🧹 Advanced data cleaning for optimization...")

# Remove any remaining NaNs and infinities
hist_returns_clean = hist_returns_clean.replace([np.inf, -np.inf], np.nan).dropna()

# Remove extreme outliers (beyond 5 standard deviations)
for col in hist_returns_clean.columns:
    col_data = hist_returns_clean[col]
    mean_val = col_data.mean()
    std_val = col_data.std()
    outlier_mask = np.abs(col_data - mean_val) > 5 * std_val
    hist_returns_clean.loc[outlier_mask, col] = np.nan

# Forward fill small gaps and drop rows with any NaNs
hist_returns_clean = hist_returns_clean.fillna(method='ffill', limit=3).dropna()

print(f"✅ Ultra-clean data: {hist_returns_clean.shape}")
print(f"📊 Data range: {hist_returns_clean.index.min()} to {hist_returns_clean.index.max()}")

# Verify data quality
print("\n🔍 Data quality check:")
print(f"   NaN values: {hist_returns_clean.isna().sum().sum()}")
print(f"   Infinite values: {np.isinf(hist_returns_clean).sum().sum()}")
print(f"   Mean return range: {hist_returns_clean.mean().min():.4f} to {hist_returns_clean.mean().max():.4f}")
print(f"   Volatility range: {hist_returns_clean.std().min():.4f} to {hist_returns_clean.std().max():.4f}")

if len(valid_symbols) >= 5 and len(hist_returns_clean) >= 50:
    print(f"\n🚀 Proceeding with portfolio optimization using {len(valid_symbols)} symbols...")
    
    # 1. Equal Weight Portfolio
    equal_weights = pd.Series(1.0/len(valid_symbols), index=valid_symbols)
    
    # Calculate robust covariance matrix
    print("📊 Calculating robust covariance matrix...")
    
    # Method 1: Simple sample covariance with regularization
    sample_cov = hist_returns_clean.cov() * 252  # Annualized
    
    # Add small regularization to diagonal to ensure positive definiteness
    regularization = 1e-6
    cov_matrix = sample_cov + np.eye(len(sample_cov)) * regularization
    
    print(f"📊 Covariance matrix shape: {cov_matrix.shape}")
    print(f"📊 Eigenvalues range: {np.linalg.eigvals(cov_matrix).min():.6f} to {np.linalg.eigvals(cov_matrix).max():.6f}")
    
    # 2. ROBUST MEAN-VARIANCE OPTIMIZATION
    print("\n💡 Implementing robust mean-variance optimization...")
    
    try:
        import cvxpy as cp
        
        # Calculate expected returns (shrunk towards grand mean)
        raw_returns = hist_returns_clean.mean() * 252  # Annualized
        grand_mean = raw_returns.mean()
        shrinkage_factor = 0.5  # Shrink 50% towards grand mean
        expected_returns = shrinkage_factor * grand_mean + (1 - shrinkage_factor) * raw_returns
        
        print(f"📊 Expected returns range: {expected_returns.min():.3f} to {expected_returns.max():.3f}")
        
        # Optimization variables
        n = len(valid_symbols)
        w = cp.Variable(n)
        
        # Risk aversion parameter (higher = more conservative)
        risk_aversion = 5.0
        
        # Objective: maximize return - risk_aversion * variance
        portfolio_return = expected_returns.values @ w
        portfolio_variance = cp.quad_form(w, cov_matrix.values)
        objective = portfolio_return - risk_aversion * portfolio_variance
        
        # Constraints
        constraints = [
            cp.sum(w) == 1,     # Weights sum to 1
            w >= 0.01,          # Minimum 1% allocation (avoids zeros)
            w <= 0.25           # Maximum 25% allocation
        ]
        
        # Solve optimization problem
        problem = cp.Problem(cp.Maximize(objective), constraints)
        problem.solve(verbose=False, solver=cp.ECOS)
        
        if w.value is not None and problem.status == cp.OPTIMAL:
            mv_weights = pd.Series(w.value, index=valid_symbols)
            mv_weights = mv_weights / mv_weights.sum()  # Ensure exact normalization
            print("✅ Mean-variance optimization completed successfully!")
            print(f"📊 Solution status: {problem.status}")
            print(f"📊 Weight range: {mv_weights.min():.3f} to {mv_weights.max():.3f}")
        else:
            print(f"⚠️  Mean-variance optimization failed - status: {problem.status}")
            mv_weights = equal_weights.copy()
    
    except ImportError:
        print("⚠️  cvxpy not available - using analytical mean-variance solution")
        
        # Analytical solution for mean-variance (assuming risk aversion = 1)
        expected_returns = hist_returns_clean.mean() * 252
        
        try:
            # Inverse covariance matrix
            inv_cov = np.linalg.inv(cov_matrix.values)
            
            # Analytical mean-variance weights
            ones = np.ones((len(valid_symbols), 1))
            
            # Calculate optimal weights
            numerator = inv_cov @ expected_returns.values.reshape(-1, 1)
            denominator = ones.T @ inv_cov @ ones
            
            mv_weights_raw = numerator / denominator
            mv_weights = pd.Series(mv_weights_raw.flatten(), index=valid_symbols)
            mv_weights = mv_weights.clip(lower=0)  # Ensure non-negative
            mv_weights = mv_weights / mv_weights.sum()  # Normalize
            
            print("✅ Analytical mean-variance optimization completed!")
        except np.linalg.LinAlgError:
            print("⚠️  Covariance matrix not invertible - using equal weights")
            mv_weights = equal_weights.copy()
    
    except Exception as e:
        print(f"⚠️  Mean-variance optimization failed: {e}")
        mv_weights = equal_weights.copy()
    
    # 3. Risk Parity Portfolio (manual implementation)
    try:
        print("\n🔧 Calculating risk parity weights...")
        # Simple risk parity: inverse volatility weighting
        vol_weights = 1.0 / hist_returns_clean.std()
        rp_weights = vol_weights / vol_weights.sum()
        print("✅ Risk parity optimization completed")
    except Exception as e:
        print(f"⚠️  Risk parity optimization failed: {e}")
        rp_weights = equal_weights.copy()
    
    # 4. Minimum Variance Portfolio
    try:
        print("🔧 Calculating minimum variance weights...")
        import cvxpy as cp
        
        n = len(valid_symbols)
        w = cp.Variable(n)
        
        # Objective: minimize portfolio variance
        portfolio_variance = cp.quad_form(w, cov_matrix.values)
        
        # Constraints
        constraints = [
            cp.sum(w) == 1,  # Weights sum to 1
            w >= 0,          # Long-only
            w <= 0.3         # Max 30% per position
        ]
        
        # Solve
        problem = cp.Problem(cp.Minimize(portfolio_variance), constraints)
        problem.solve(verbose=False)
        
        if w.value is not None:
            mv_min_weights = pd.Series(w.value, index=valid_symbols)
            print("✅ Minimum variance optimization completed")
        else:
            mv_min_weights = equal_weights.copy()
            print("⚠️  Minimum variance optimization failed - using equal weights")
    
    except Exception as e:
        print(f"⚠️  Minimum variance optimization failed: {e}")
        mv_min_weights = equal_weights.copy()
    
    # Store all optimization results
    optimization_results = {
        'Equal Weight': equal_weights,
        'Mean Variance': mv_weights,
        'Risk Parity': rp_weights,
        'Minimum Variance': mv_min_weights
    }
    
    # Display portfolio characteristics
    print("\n📊 Portfolio Characteristics:")
    print("=" * 70)
    
    for name, weights in optimization_results.items():
        if len(weights) > 0 and not weights.isna().all():
            # Calculate portfolio metrics
            portfolio_return = (weights * hist_returns_clean.mean() * 252).sum()
            portfolio_vol = np.sqrt(weights.T @ cov_matrix @ weights)
            sharpe_ratio = portfolio_return / portfolio_vol if portfolio_vol > 0 else 0
            max_weight = weights.max()
            concentration = (weights ** 2).sum()  # Herfindahl index
            
            print(f"\n{name}:")
            print(f"   Expected Return: {portfolio_return:.2%}")
            print(f"   Volatility: {portfolio_vol:.2%}")
            print(f"   Sharpe Ratio: {sharpe_ratio:.3f}")
            print(f"   Max Weight: {max_weight:.2%}")
            print(f"   Concentration: {concentration:.3f}")
            
            # Show top holdings with percentages
            top_holdings = weights.nlargest(3)
            top_holdings_dict = {symbol: f"{weight:.1%}" for symbol, weight in top_holdings.items()}
            print(f"   Top 3 holdings: {top_holdings_dict}")
            
            # Check if this portfolio is different from equal weight
            if name == 'Mean Variance':
                weight_diff = np.abs(weights - equal_weights).sum()
                print(f"   📏 Difference from Equal Weight: {weight_diff:.4f}")
                if weight_diff < 0.01:
                    print("   ⚠️  WARNING: Very similar to equal weight - optimization may not be working")
                else:
                    print("   ✅ Successfully differentiated from equal weight")
    
    # VERIFICATION: Show weight comparison
    print("\n🔍 Weight Comparison (Top 5 positions):")
    print("=" * 70)
    comparison_df = pd.DataFrame({
        name: weights.nlargest(5) for name, weights in optimization_results.items()
    })
    print(comparison_df.round(4))
    
else:
    print("❌ Insufficient data for meaningful portfolio optimization")

💼 Comparing portfolio optimization methods...
📅 Optimization date: 2023-10-05
📊 Historical data: 347 observations
📊 Available symbols: 36
✅ Valid symbols for optimization: 36
📊 Clean data shape: (347, 36)
📊 Using top 15 symbols for optimization

🧹 Advanced data cleaning for optimization...
✅ Ultra-clean data: (347, 15)
📊 Data range: 2022-05-19 00:00:00 to 2023-10-05 00:00:00

🔍 Data quality check:
   NaN values: 0
   Infinite values: 0
   Mean return range: -0.0007 to 0.0020
   Volatility range: 0.0101 to 0.0241

🚀 Proceeding with portfolio optimization using 15 symbols...
📊 Calculating robust covariance matrix...
📊 Covariance matrix shape: (15, 15)
📊 Eigenvalues range: 0.009713 to 0.612701

💡 Implementing robust mean-variance optimization...
📊 Expected returns range: -0.045 to 0.296
✅ Mean-variance optimization completed successfully!
📊 Solution status: optimal
📊 Weight range: 0.010 to 0.250

🔧 Calculating risk parity weights...
✅ Risk parity optimization completed
🔧 Calculating minim

## 4. Risk Budgeting and Factor Allocation

In [6]:
print("🎯 Risk budgeting analysis...")

def calculate_risk_contributions(weights, cov_matrix):
    """
    Calculate marginal and component risk contributions.
    """
    portfolio_vol = np.sqrt(weights.T @ cov_matrix @ weights)
    marginal_contrib = (cov_matrix @ weights) / portfolio_vol
    contrib = weights * marginal_contrib
    return contrib / contrib.sum(), marginal_contrib

# Analyze risk contributions for each portfolio
risk_analysis = {}

for name, weights in optimization_results.items():
    if len(weights) > 0 and not weights.isna().all():
        # Ensure weights are aligned with covariance matrix
        aligned_weights = weights.reindex(cov_matrix.index).fillna(0)
        
        if aligned_weights.sum() > 0.5:  # Valid portfolio
            risk_contrib, marginal_contrib = calculate_risk_contributions(
                aligned_weights, cov_matrix
            )
            
            risk_analysis[name] = {
                'weights': aligned_weights,
                'risk_contrib': risk_contrib,
                'marginal_contrib': marginal_contrib
            }

# Display risk contribution analysis
print("\n📊 Risk Contribution Analysis:")
print("=" * 40)

for name, analysis in risk_analysis.items():
    weights = analysis['weights']
    risk_contrib = analysis['risk_contrib']
    
    print(f"\n{name}:")
    
    # Top risk contributors
    top_risk = risk_contrib.nlargest(5)
    print(f"   Top risk contributors:")
    for symbol, contrib in top_risk.items():
        weight = weights[symbol]
        print(f"     {symbol}: {contrib:.2%} risk (weight: {weight:.2%})")
    
    # Sector risk breakdown
    sector_risk = {}
    sector_weights = {}
    
    for sector, sector_symbols in universe.items():
        sector_contrib = 0
        sector_weight = 0
        
        for symbol in sector_symbols:
            if symbol in risk_contrib.index:
                sector_contrib += risk_contrib[symbol]
                sector_weight += weights[symbol]
        
        if sector_contrib > 0:
            sector_risk[sector] = sector_contrib
            sector_weights[sector] = sector_weight
    
    print(f"   Sector risk allocation:")
    for sector in sorted(sector_risk.keys(), key=lambda x: sector_risk[x], reverse=True):
        print(f"     {sector}: {sector_risk[sector]:.2%} risk (weight: {sector_weights[sector]:.2%})")

print(f"\n✅ Risk budgeting analysis completed")

🎯 Risk budgeting analysis...

📊 Risk Contribution Analysis:

Equal Weight:
   Top risk contributors:
     EOG: 9.26% risk (weight: 6.67%)
     COP: 8.86% risk (weight: 6.67%)
     BA: 8.40% risk (weight: 6.67%)
     CAT: 8.30% risk (weight: 6.67%)
     C: 7.67% risk (weight: 6.67%)
   Sector risk allocation:
     Industrial: 29.15% risk (weight: 26.67%)
     Energy: 24.91% risk (weight: 20.00%)
     Finance: 22.50% risk (weight: 20.00%)
     Technology: 13.83% risk (weight: 13.33%)
     Consumer: 5.04% risk (weight: 6.67%)
     Healthcare: 4.58% risk (weight: 13.33%)

Mean Variance:
   Top risk contributors:
     GE: 35.34% risk (weight: 25.00%)
     ABBV: 19.73% risk (weight: 25.00%)
     JNJ: 13.55% risk (weight: 23.44%)
     BA: 9.27% risk (weight: 6.43%)
     AAPL: 6.75% risk (weight: 5.91%)
   Sector risk allocation:
     Industrial: 48.68% risk (weight: 35.05%)
     Healthcare: 33.28% risk (weight: 48.44%)
     Technology: 9.43% risk (weight: 8.27%)
     Energy: 4.40% risk (weigh

## 5. Dynamic Hedging Strategy

In [7]:
print("🛡️  Building dynamic hedging strategy...")

# Define hedging instruments (tradeable ETFs)
hedge_instruments = ['VIXY', 'TLT', 'GLD', 'UUP', 'SHY', 'VXX']  # Vol ETF, Long Bonds, Gold, USD, Short Bonds, Vol ETF

print(f"📊 Attempting to load hedge instruments: {hedge_instruments}")

# Fetch hedge instrument data with robust error handling
hedge_data = pd.DataFrame()
successful_instruments = []

for instrument in hedge_instruments:
    try:
        instrument_data = data_manager.get_prices(
            symbols=[instrument],
            start_date="2020-01-01",
            end_date="2024-01-01"
        )
        if not instrument_data.empty and len(instrument_data) > 100:  # Require sufficient data
            hedge_data[instrument] = instrument_data[instrument]
            successful_instruments.append(instrument)
            print(f"✅ {instrument}: {len(instrument_data)} observations loaded")
        else:
            print(f"⚠️  {instrument}: Insufficient data ({len(instrument_data)} observations)")
    except Exception as e:
        print(f"❌ {instrument}: Failed to load - {e}")

if not hedge_data.empty:
    hedge_returns = hedge_data.pct_change().dropna()
    print(f"\n✅ Hedge instrument data loaded: {len(hedge_returns)} observations")
    print(f"📊 Available hedge instruments: {hedge_returns.columns.tolist()}")
    print(f"📅 Date range: {hedge_returns.index.min()} to {hedge_returns.index.max()}")
else:
    print("⚠️  No hedge instruments could be loaded - proceeding with portfolio-only analysis")
    hedge_returns = pd.DataFrame()

# Build dynamic hedging model
def calculate_hedge_ratios(portfolio_returns, hedge_returns, lookback=60):
    """
    Calculate optimal hedge ratios using rolling regression.
    """
    hedge_ratios = pd.DataFrame(index=portfolio_returns.index)
    
    if hedge_returns.empty:
        return hedge_ratios
    
    for date in portfolio_returns.index[lookback:]:
        # Get historical data
        hist_portfolio = portfolio_returns.loc[date-pd.Timedelta(days=lookback*2):date]
        hist_hedge = hedge_returns.loc[date-pd.Timedelta(days=lookback*2):date]
        
        # Align data
        common_dates = hist_portfolio.index.intersection(hist_hedge.index)
        if len(common_dates) >= 30:  # Minimum observations
            aligned_portfolio = hist_portfolio.loc[common_dates]
            aligned_hedge = hist_hedge.loc[common_dates]
            
            # Calculate hedge ratios using regression
            for hedge_asset in hedge_returns.columns:
                if hedge_asset in aligned_hedge.columns:
                    # Simple correlation-based hedge ratio
                    correlation = aligned_portfolio.corr(aligned_hedge[hedge_asset])
                    volatility_ratio = aligned_hedge[hedge_asset].std() / aligned_portfolio.std()
                    
                    if not pd.isna(correlation) and not pd.isna(volatility_ratio):
                        hedge_ratio = -correlation * volatility_ratio  # Negative for hedging
                        hedge_ratios.loc[date, hedge_asset] = hedge_ratio
    
    return hedge_ratios

# Create a sample portfolio return series for hedging analysis
if len(valid_symbols) > 0 and 'Mean Variance' in optimization_results:
    mv_weights = optimization_results['Mean Variance']
    sample_portfolio_returns = (returns[valid_symbols] * mv_weights).sum(axis=1)
    
    print(f"📊 Sample portfolio created with {len(valid_symbols)} assets")
    
    # Calculate hedge ratios
    if not hedge_returns.empty:
        print(f"\n🔧 Calculating hedge ratios...")
        hedge_ratios = calculate_hedge_ratios(sample_portfolio_returns, hedge_returns)
        
        print(f"🛡️  Hedge ratios calculated for {len(hedge_ratios.columns)} instruments")
        
        # Display average hedge ratios
        if not hedge_ratios.empty and len(hedge_ratios.columns) > 0:
            print("\n📋 Average hedge ratios (negative = short position for hedging):")
            for instrument in hedge_ratios.columns:
                avg_ratio = hedge_ratios[instrument].mean()
                if not pd.isna(avg_ratio):
                    hedge_direction = "SHORT" if avg_ratio < 0 else "LONG"
                    print(f"   {instrument}: {avg_ratio:.3f} ({hedge_direction})")
            
            # Show hedge ratio evolution over time
            if len(hedge_ratios) > 50:  # Sufficient data for analysis
                print(f"\n📈 Hedge ratio statistics:")
                print(f"   Data points: {len(hedge_ratios)}")
                print(f"   Coverage: {hedge_ratios.index.min().strftime('%Y-%m-%d')} to {hedge_ratios.index.max().strftime('%Y-%m-%d')}")
                
                for instrument in hedge_ratios.columns:
                    ratios = hedge_ratios[instrument].dropna()
                    if len(ratios) > 10:
                        print(f"   {instrument}: μ={ratios.mean():.3f}, σ={ratios.std():.3f}, range=[{ratios.min():.3f}, {ratios.max():.3f}]")
        else:
            print("⚠️  No hedge ratios could be calculated - insufficient overlapping data")
    else:
        print("⚠️  No hedge instruments available for ratio calculation")
    
    # Risk-adjusted portfolio performance
    portfolio_vol = sample_portfolio_returns.std() * np.sqrt(252)
    portfolio_return = sample_portfolio_returns.mean() * 252
    portfolio_sharpe = portfolio_return / portfolio_vol if portfolio_vol > 0 else 0
    
    print(f"\n📊 Portfolio performance metrics:")
    print(f"   Annualized Return: {portfolio_return:.2%}")
    print(f"   Annualized Volatility: {portfolio_vol:.2%}")
    print(f"   Sharpe Ratio: {portfolio_sharpe:.2f}")
    
    # Calculate maximum drawdown
    cumulative_returns = (1 + sample_portfolio_returns).cumprod()
    rolling_max = cumulative_returns.expanding().max()
    drawdown = (cumulative_returns - rolling_max) / rolling_max
    max_drawdown = drawdown.min()
    
    print(f"   Maximum Drawdown: {max_drawdown:.2%}")
    
else:
    print("⚠️  Insufficient data for hedging analysis")

🛡️  Building dynamic hedging strategy...
📊 Attempting to load hedge instruments: ['VIXY', 'TLT', 'GLD', 'UUP', 'SHY', 'VXX']
✅ VIXY: 1006 observations loaded✅ VIXY: 1006 observations loaded

✅ TLT: 1006 observations loaded
✅ TLT: 1006 observations loaded
✅ GLD: 1006 observations loaded
✅ GLD: 1006 observations loaded
✅ UUP: 1006 observations loaded
✅ UUP: 1006 observations loaded
✅ SHY: 1006 observations loaded
✅ SHY: 1006 observations loaded
✅ VXX: 1006 observations loaded

✅ Hedge instrument data loaded: 1005 observations
📊 Available hedge instruments: ['VIXY', 'TLT', 'GLD', 'UUP', 'SHY', 'VXX']
📅 Date range: 2020-01-03 00:00:00 to 2023-12-29 00:00:00
📊 Sample portfolio created with 15 assets

🔧 Calculating hedge ratios...
✅ VXX: 1006 observations loaded

✅ Hedge instrument data loaded: 1005 observations
📊 Available hedge instruments: ['VIXY', 'TLT', 'GLD', 'UUP', 'SHY', 'VXX']
📅 Date range: 2020-01-03 00:00:00 to 2023-12-29 00:00:00
📊 Sample portfolio created with 15 assets

🔧 Calcu

## 6. Stress Testing and Scenario Analysis

In [8]:
print("🔬 Stress testing and scenario analysis...")

def stress_test_portfolio(weights, returns, scenarios):
    """
    Perform stress testing on portfolio under various scenarios.
    """
    results = {}
    
    # Align weights with returns
    aligned_weights = weights.reindex(returns.columns).fillna(0)
    
    for scenario_name, scenario_shocks in scenarios.items():
        scenario_returns = returns.copy()
        
        # Apply shocks
        for asset, shock in scenario_shocks.items():
            if asset in scenario_returns.columns:
                scenario_returns[asset] += shock
        
        # Calculate portfolio impact
        portfolio_impact = (scenario_returns * aligned_weights).sum(axis=1)
        
        results[scenario_name] = {
            'portfolio_return': portfolio_impact.sum(),
            'worst_day': portfolio_impact.min(),
            'volatility': portfolio_impact.std(),
            'var_95': portfolio_impact.quantile(0.05),
            'var_99': portfolio_impact.quantile(0.01)
        }
    
    return results

# Define stress test scenarios
stress_scenarios = {
    'Market Crash (-20%)': {symbol: -0.20 for symbol in all_symbols},
    'Tech Selloff': {symbol: -0.30 for symbol in universe['Technology']},
    'Financial Crisis': {symbol: -0.40 for symbol in universe['Finance']},
    'Energy Shock': {symbol: 0.50 for symbol in universe['Energy']},  # Oil spike
    'Flight to Quality': {
        **{symbol: -0.15 for symbol in universe['Technology']},
        **{symbol: 0.05 for symbol in universe['Consumer']}
    }
}

# Run stress tests for each portfolio
print("\n🔬 Stress Test Results:")
print("=" * 30)

for portfolio_name, weights in optimization_results.items():
    if len(weights) > 0 and not weights.isna().all():
        print(f"\n{portfolio_name}:")
        
        # Use a subset of recent returns for stress testing
        recent_returns = returns.tail(252)  # Last year
        stress_results = stress_test_portfolio(weights, recent_returns, stress_scenarios)
        
        for scenario, metrics in stress_results.items():
            print(f"\n   {scenario}:")
            print(f"     Portfolio Return: {metrics['portfolio_return']:.2%}")
            print(f"     Worst Single Day: {metrics['worst_day']:.2%}")
            print(f"     VaR (95%): {metrics['var_95']:.2%}")
            print(f"     VaR (99%): {metrics['var_99']:.2%}")

# Historical scenario analysis (actual market events)
print("\n📊 Historical Scenario Analysis:")
print("=" * 35)

# Define historical periods of interest
historical_periods = {
    'COVID Crash (Mar 2020)': ('2020-02-19', '2020-03-23'),
    'Post-COVID Recovery (Apr-Jun 2020)': ('2020-04-01', '2020-06-30'),
    'Tech Bubble Peak (2021)': ('2021-01-01', '2021-12-31'),
    'Rising Rates (2022)': ('2022-01-01', '2022-12-31'),
    'Banking Crisis (Mar 2023)': ('2023-03-01', '2023-03-31')
}

for portfolio_name, weights in optimization_results.items():
    if len(weights) > 0 and not weights.isna().all():
        print(f"\n{portfolio_name}:")
        
        aligned_weights = weights.reindex(returns.columns).fillna(0)
        
        for period_name, (start_date, end_date) in historical_periods.items():
            try:
                period_returns = returns.loc[start_date:end_date]
                if len(period_returns) > 0:
                    portfolio_period_returns = (period_returns * aligned_weights).sum(axis=1)
                    
                    total_return = (1 + portfolio_period_returns).prod() - 1
                    volatility = portfolio_period_returns.std() * np.sqrt(252)
                    worst_day = portfolio_period_returns.min()
                    
                    print(f"   {period_name}:")
                    print(f"     Total Return: {total_return:.2%}")
                    print(f"     Annualized Vol: {volatility:.2%}")
                    print(f"     Worst Day: {worst_day:.2%}")
            
            except Exception as e:
                continue

print("\n✅ Stress testing completed")

🔬 Stress testing and scenario analysis...

🔬 Stress Test Results:

Equal Weight:

   Market Crash (-20%):
     Portfolio Return: -5020.27%
     Worst Single Day: -22.29%
     VaR (95%): -21.46%
     VaR (99%): -21.90%

   Tech Selloff:
     Portfolio Return: -988.27%
     Worst Single Day: -6.29%
     VaR (95%): -5.46%
     VaR (99%): -5.90%

   Financial Crisis:
     Portfolio Return: -1996.27%
     Worst Single Day: -10.29%
     VaR (95%): -9.46%
     VaR (99%): -9.90%

   Energy Shock:
     Portfolio Return: 2539.73%
     Worst Single Day: 7.71%
     VaR (95%): 8.54%
     VaR (99%): 8.10%

   Flight to Quality:
     Portfolio Return: -400.27%
     Worst Single Day: -3.96%
     VaR (95%): -3.13%
     VaR (99%): -3.57%

Mean Variance:

   Market Crash (-20%):
     Portfolio Return: -5015.64%
     Worst Single Day: -22.33%
     VaR (95%): -21.21%
     VaR (99%): -21.83%

   Tech Selloff:
     Portfolio Return: -601.02%
     Worst Single Day: -4.81%
     VaR (95%): -3.69%
     VaR (99%)

## 7. Portfolio Risk Attribution

In [9]:
print("📊 Portfolio risk attribution analysis...")

def decompose_portfolio_risk(weights, factor_exposures, factor_returns):
    """
    Decompose portfolio risk into factor and specific components with proper normalization.
    """
    # Calculate portfolio factor exposures
    portfolio_exposures = {}
    
    factor_types = ['market_beta', 'size', 'momentum', 'volatility']
    
    for factor_type in factor_types:
        factor_values = []
        factor_weights = []
        
        for symbol, weight in weights.items():
            if weight > 0.001:  # Active positions only
                exposure_col = f"{symbol}_{factor_type}"
                if exposure_col in factor_exposures.columns:
                    latest_exposure = factor_exposures[exposure_col].iloc[-1]
                    if not pd.isna(latest_exposure) and not np.isinf(latest_exposure):
                        factor_values.append(latest_exposure)
                        factor_weights.append(weight)
        
        if len(factor_values) > 0:
            # Calculate weighted average
            portfolio_exposures[factor_type] = np.average(factor_values, weights=factor_weights)
        else:
            portfolio_exposures[factor_type] = 0
    
    return portfolio_exposures

def calculate_factor_statistics(factor_exposures, factor_types):
    """
    Calculate cross-sectional statistics for factor normalization.
    """
    factor_stats = {}
    
    for factor_type in factor_types:
        factor_cols = [col for col in factor_exposures.columns if col.endswith(f'_{factor_type}')]
        
        if len(factor_cols) > 0:
            # Get latest cross-sectional data
            latest_values = []
            for col in factor_cols:
                val = factor_exposures[col].iloc[-1]
                if not pd.isna(val) and not np.isinf(val):
                    latest_values.append(val)
            
            if len(latest_values) > 2:
                factor_stats[factor_type] = {
                    'mean': np.mean(latest_values),
                    'std': np.std(latest_values),
                    'median': np.median(latest_values),
                    'q25': np.percentile(latest_values, 25),
                    'q75': np.percentile(latest_values, 75)
                }
    
    return factor_stats

# Calculate factor statistics for interpretation
factor_stats = calculate_factor_statistics(factor_exposures, ['market_beta', 'size', 'momentum', 'volatility'])

# Analyze risk attribution for each portfolio
print("\n🎯 Risk Attribution Results:")
print("=" * 35)

for portfolio_name, weights in optimization_results.items():
    if len(weights) > 0 and not weights.isna().all():
        print(f"\n{portfolio_name}:")
        
        # Portfolio factor exposures
        exposures = decompose_portfolio_risk(weights, factor_exposures, returns)
        
        print(f"   Factor Exposures:")
        for factor, exposure in exposures.items():
            if factor in factor_stats:
                stats = factor_stats[factor]
                
                # Calculate z-score for interpretation
                if stats['std'] > 0:
                    z_score = (exposure - stats['mean']) / stats['std']
                    percentile = ((exposure - stats['median']) / (stats['q75'] - stats['q25'])) * 50 + 50
                    percentile = max(0, min(100, percentile))  # Bound between 0-100
                    
                    if factor == 'momentum':
                        print(f"     {factor:12}: {exposure:6.1f}% (z={z_score:+.1f}, {percentile:.0f}th percentile)")
                    elif factor == 'volatility':
                        print(f"     {factor:12}: {exposure:6.1f}% (z={z_score:+.1f}, {percentile:.0f}th percentile)")
                    else:
                        print(f"     {factor:12}: {exposure:6.3f} (z={z_score:+.1f}, {percentile:.0f}th percentile)")
                else:
                    if factor == 'momentum':
                        print(f"     {factor:12}: {exposure:6.1f}%")
                    elif factor == 'volatility':
                        print(f"     {factor:12}: {exposure:6.1f}%")
                    else:
                        print(f"     {factor:12}: {exposure:6.3f}")
            else:
                print(f"     {factor:12}: {exposure:6.3f}")
        
        # Sector allocation
        sector_allocation = {}
        for sector, sector_symbols in universe.items():
            sector_weight = 0
            for symbol in sector_symbols:
                if symbol in weights.index:
                    sector_weight += weights[symbol]
            sector_allocation[sector] = sector_weight
        
        print(f"   Sector Allocation:")
        for sector, allocation in sorted(sector_allocation.items(), key=lambda x: x[1], reverse=True):
            if allocation > 0.001:
                print(f"     {sector:12}: {allocation:6.2%}")
        
        # Risk concentration metrics
        herfindahl_index = (weights ** 2).sum()
        effective_positions = 1 / herfindahl_index if herfindahl_index > 0 else 0
        
        print(f"   Risk Concentration:")
        print(f"     Herfindahl Index: {herfindahl_index:.3f}")
        print(f"     Effective Positions: {effective_positions:.1f}")
        print(f"     Max Position: {weights.max():.2%}")
        print(f"     Top 5 Concentration: {weights.nlargest(5).sum():.2%}")

print("\n✅ Risk attribution analysis completed")

📊 Portfolio risk attribution analysis...

🎯 Risk Attribution Results:

Equal Weight:
   Factor Exposures:
     market_beta :  0.953 (z=+0.2, 55th percentile)
     size        :  0.829 (z=+0.3, 41th percentile)
     momentum    :   21.3% (z=-0.1, 68th percentile)
     volatility  :    0.2% (z=-0.0, 46th percentile)
   Sector Allocation:
     Industrial  : 26.67%
     Finance     : 20.00%
     Energy      : 20.00%
     Technology  : 13.33%
     Healthcare  : 13.33%
     Consumer    :  6.67%
   Risk Concentration:
     Herfindahl Index: 0.067
     Effective Positions: 15.0
     Max Position: 6.67%
     Top 5 Concentration: 33.33%

Mean Variance:
   Factor Exposures:
     market_beta :  0.803 (z=-0.1, 44th percentile)
     size        :  0.918 (z=+0.6, 52th percentile)
     momentum    :   31.2% (z=+0.1, 82th percentile)
     volatility  :    0.2% (z=-0.5, 33th percentile)
   Sector Allocation:
     Healthcare  : 48.44%
     Industrial  : 35.05%
     Technology  :  8.27%
     Energy      :

## 8. Visualization and Summary

In [11]:
print("📊 Creating risk management visualizations...")

# Create figures directory
figures_dir = Path("../data/figures")
figures_dir.mkdir(parents=True, exist_ok=True)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Portfolio comparison chart
fig1 = go.Figure()

# Calculate and plot efficient frontier
if len(valid_symbols) >= 5:
    vol_range = np.linspace(0.10, 0.40, 20)
    efficient_returns = []
    efficient_vols = []
    
    for target_vol in vol_range:
        try:
            # Simple mean-variance optimization for efficient frontier
            expected_returns = hist_returns_clean.mean() * 252
            cov_matrix_annual = hist_returns_clean.cov() * 252
            
            import cvxpy as cp
            w = cp.Variable(len(valid_symbols))
            
            # Maximize return for given volatility
            portfolio_return = w.T @ expected_returns
            portfolio_variance = cp.quad_form(w, cov_matrix_annual)
            
            constraints = [
                cp.sum(w) == 1,
                w >= 0,
                cp.sqrt(portfolio_variance) <= target_vol
            ]
            
            problem = cp.Problem(cp.Maximize(portfolio_return), constraints)
            problem.solve()
            
            if w.value is not None:
                efficient_returns.append(portfolio_return.value)
                efficient_vols.append(target_vol)
        
        except:
            continue
    
    # Plot efficient frontier
    if len(efficient_returns) > 5:
        fig1.add_trace(go.Scatter(
            x=efficient_vols,
            y=efficient_returns,
            mode='lines',
            name='Efficient Frontier',
            line=dict(color='gray', dash='dash')
        ))

# Plot portfolio points
colors = ['blue', 'red', 'green', 'orange']
for i, (name, weights) in enumerate(optimization_results.items()):
    if len(weights) > 0 and not weights.isna().all():
        aligned_weights = weights.reindex(hist_returns_clean.columns).fillna(0)
        
        if aligned_weights.sum() > 0.5:  # Valid portfolio
            portfolio_return = (aligned_weights * hist_returns_clean.mean() * 252).sum()
            portfolio_vol = np.sqrt(aligned_weights.T @ cov_matrix @ aligned_weights)
            
            fig1.add_trace(go.Scatter(
                x=[portfolio_vol],
                y=[portfolio_return],
                mode='markers',
                name=name,
                marker=dict(size=12, color=colors[i % len(colors)])
            ))

fig1.update_layout(
    title="Portfolio Optimization Comparison",
    xaxis_title="Volatility (Annual)",
    yaxis_title="Expected Return (Annual)",
    template="plotly_white",
    width=800,
    height=600
)

fig1.show()
fig1.write_html(str(figures_dir / "portfolio_optimization_comparison.html"))
print(f"💾 Saved portfolio comparison to: {figures_dir / 'portfolio_optimization_comparison.html'}")

# 2. Risk contribution breakdown
if len(risk_analysis) > 0:
    fig2 = make_subplots(
        rows=2, cols=2,
        subplot_titles=list(risk_analysis.keys()),
        specs=[[{"type": "pie"}, {"type": "pie"}],
               [{"type": "pie"}, {"type": "pie"}]]
    )
    
    positions = [(1, 1), (1, 2), (2, 1), (2, 2)]
    
    for i, (name, analysis) in enumerate(list(risk_analysis.items())[:4]):
        risk_contrib = analysis['risk_contrib']
        
        # Top 10 risk contributors
        top_contrib = risk_contrib.nlargest(10)
        other_contrib = risk_contrib[~risk_contrib.index.isin(top_contrib.index)].sum()
        
        labels = list(top_contrib.index) + ['Others']
        values = list(top_contrib.values) + [other_contrib]
        
        fig2.add_trace(go.Pie(
            labels=labels,
            values=values,
            name=name
        ), row=positions[i][0], col=positions[i][1])
    
    fig2.update_layout(
        title="Risk Contribution by Asset",
        template="plotly_white",
        height=800
    )
    
    fig2.show()
    fig2.write_html(str(figures_dir / "risk_contribution_breakdown.html"))
    print(f"💾 Saved risk breakdown to: {figures_dir / 'risk_contribution_breakdown.html'}")

print("\n✅ Risk management analysis completed!")
print("\n🎯 Key Insights:")
print("   • Compared multiple optimization methods (Mean-Variance, Risk Parity, Min Variance)")
print("   • Analyzed risk contributions and factor exposures")
print("   • Performed stress testing under various scenarios")
print("   • Evaluated portfolio performance during historical market events")
print("   • Assessed concentration risk and diversification benefits")
print("\n💡 Best Practices:")
print("   • Use multiple optimization techniques for robust portfolios")
print("   • Monitor factor exposures and risk contributions regularly")
print("   • Stress test portfolios under extreme scenarios")
print("   • Consider dynamic hedging for tail risk protection")
print("   • Balance expected returns with risk management objectives")

📊 Creating risk management visualizations...


💾 Saved portfolio comparison to: ../data/figures/portfolio_optimization_comparison.html


💾 Saved risk breakdown to: ../data/figures/risk_contribution_breakdown.html

✅ Risk management analysis completed!

🎯 Key Insights:
   • Compared multiple optimization methods (Mean-Variance, Risk Parity, Min Variance)
   • Analyzed risk contributions and factor exposures
   • Performed stress testing under various scenarios
   • Evaluated portfolio performance during historical market events
   • Assessed concentration risk and diversification benefits

💡 Best Practices:
   • Use multiple optimization techniques for robust portfolios
   • Monitor factor exposures and risk contributions regularly
   • Stress test portfolios under extreme scenarios
   • Consider dynamic hedging for tail risk protection
   • Balance expected returns with risk management objectives


## Summary

This notebook demonstrated advanced risk management and portfolio optimization techniques:

### Portfolio Optimization Methods
- **Equal Weight**: Simple diversification baseline
- **Mean-Variance**: Maximizes risk-adjusted returns
- **Risk Parity**: Equalizes risk contributions
- **Minimum Variance**: Minimizes portfolio volatility

### Risk Management Framework
- **Factor Risk Model**: Market, size, momentum, volatility factors
- **Risk Attribution**: Component and marginal risk contributions
- **Stress Testing**: Scenario analysis and historical simulations
- **Dynamic Hedging**: Correlation-based hedge ratios

### Key Metrics
- Expected return and volatility
- Sharpe ratio and Calmar ratio
- Maximum drawdown and VaR
- Concentration risk (Herfindahl index)
- Factor exposures and sector allocation

### Applications
- Institutional portfolio management
- Risk budgeting and allocation
- Regulatory capital requirements
- Performance attribution analysis
- Tail risk hedging strategies